### **Environment Introduction** 

### **Taxi V3 Gym OPENAI**

In [1]:
import gym

env = gym.make("Taxi-v3").env

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### **Random Initialzed rewards of each action of a state**
**Brute Force Approach**

**State chosen = 10**

In [2]:
env.P[10]

{0: [(1.0, 110, -1, False)],
 1: [(1.0, 10, -1, False)],
 2: [(1.0, 30, -1, False)],
 3: [(1.0, 10, -1, False)],
 4: [(1.0, 10, -10, False)],
 5: [(1.0, 10, -10, False)]}

In [3]:
env.s = 10
epochs = 0
pen, rew = 0, 0

frames = [] # for results

done = False

while not done:
    action = env.action_space.sample()
    state, rew, done, info = env.step(action)

    if rew == -10:
        pen += 1
    
    # Put each rendered frame into dict for animation
    frames.append({'frame': env.render(mode='ansi'),'state': state,'action': action,'reward': rew})
    epochs += 1
  
print("Time_steps taken: {}".format(epochs))
print("Penalties: {}".format(pen))


Time_steps taken: 4614
Penalties: 1413


In [19]:
import time
from IPython.display import clear_output
def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        time.sleep(0.1)
print_frames(frames)


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 4614
State: 410
Action: 5
Reward: 20


In [20]:
from statistics import mean
mean(d['reward'] for d in frames if d)

-3.7516254876462938

# **Training a RL agent**

In [0]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [22]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1
episodes = 100001

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, episodes):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.



# **Agent Evaluation**

In [25]:
#Agent Eval
total_epochs, total_penalties,total_rewards = 0, 0,0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_rewards += reward
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average rewards per episode: {total_rewards / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.79
Average rewards per episode: 20.0
Average penalties per episode: 0.0
